### Simple Wrappers

In [1]:
import inspect
from pprint import pprint
from os.path import dirname
from CybORG import CybORG
from CybORG.Agents.Wrappers import *
from CybORG.Simulator.Scenarios import FileReaderScenarioGenerator

path = inspect.getfile(CybORG)
path = dirname(path) + f'/Simulator/Scenarios/scenario_files/Scenario1b.yaml'
sg = FileReaderScenarioGenerator(path)

cyborg = CybORG(scenario_generator=sg)

Our demonstrations have mostly been with native CybORG, but we will now demonstrate how AI can be assisted by the use of Wrappers.

Our first Wrapper is EnumActionWrapper, which calculates all the possible actions and returns the action space as the number of such actions. This gives a relatively large number of actions for both Blue and Red team.

In [2]:
env = EnumActionWrapper(cyborg)

results = env.reset(agent='Blue')
action_space = results.action_space
print('Blue action space:',action_space)

results = env.reset(agent='Red')
action_space = results.action_space
print('Red action space:', action_space)

Blue action space: 66
Red action space: 68


We can do a similar thing for the observation space. The FlatFixedWrapper parses the internal state of CybORG and turns it into a list of floats, which can easily be converted into a vector. Unfortunately, this vector is extremely long, with length over 11,000!

In [3]:
env = FixedFlatWrapper(cyborg)

results = env.reset()
obs = results.observation
print(type(obs))
print(len(obs))

<class 'list'>
11293


The OpenAIGymWrapper converts the output of FlatFixedWrapper to a numpy array as well as conforming to other parts of the OpenAI Gym API. It requires FlatFixedWrapper in order to function and should always be the outermost of the provided wrappers. Note it also performs the functions of the EnumActionWrapper You must also specify an agent parameter and explitly specify the environment parameter.

In [4]:
wrappers = FixedFlatWrapper(cyborg)
env = OpenAIGymWrapper(env=wrappers,agent_name='Blue')

obs = env.reset()
print('Observation:',obs)
print(73*'-')
print('Action_Space:',env.action_space)
print(73*'-')
print('Observation Space:',env.observation_space)

Observation: [ 0.6666667  0.         1.        ... -1.        -1.        -1.       ]
-------------------------------------------------------------------------
Action_Space: Discrete(66)
-------------------------------------------------------------------------
Observation Space: Box([-1. -1. -1. ... -1. -1. -1.], [3. 3. 3. ... 3. 3. 3.], (11293,), float32)


### Table Wrappers

The 'Table Wrappers' attempt to use basic logic to infer a human-friendly picture of the state by keeping track of past observations. This allows for a greatly simplified state space and much greater human readibility. However, it mercilessly exploits the current limitations of Scenario 1b and thus would have limited use on real-world cyber problems.

The first wrapper is the TrueTableWrapper, which modifies the get_agent_state method to return the true state in the form of the table.

In [5]:
env = TrueTableWrapper(cyborg)

env.reset()

true_table = env.get_agent_state('True')
print(true_table)

+-----------------+--------------+-------------+-------+---------+------------+
|      Subnet     |  IP Address  |   Hostname  | Known | Scanned |   Access   |
+-----------------+--------------+-------------+-------+---------+------------+
| 10.0.231.208/28 | 10.0.231.209 |   Defender  | False |  False  |    None    |
| 10.0.231.208/28 | 10.0.231.218 | Enterprise0 | False |  False  |    None    |
| 10.0.231.208/28 | 10.0.231.215 | Enterprise1 | False |  False  |    None    |
| 10.0.231.208/28 | 10.0.231.220 | Enterprise2 | False |  False  |    None    |
| 10.0.131.208/28 | 10.0.131.221 |   Op_Host0  | False |  False  |    None    |
| 10.0.131.208/28 | 10.0.131.220 |   Op_Host1  | False |  False  |    None    |
| 10.0.131.208/28 | 10.0.131.210 |   Op_Host2  | False |  False  |    None    |
| 10.0.131.208/28 | 10.0.131.218 |  Op_Server0 | False |  False  |    None    |
|  10.0.159.16/28 | 10.0.159.30  |    User0    |  True |  False  | Privileged |
|  10.0.159.16/28 | 10.0.159.21  |    Us

The BlueTableWrapper provides similar functionality for the blue observation.

In [6]:
from CybORG.Agents import B_lineAgent
from CybORG.Simulator.Actions import Sleep
cyborg = CybORG(sg, agents={'Red':B_lineAgent()})

env = BlueTableWrapper(cyborg)

results = env.reset(agent='Blue')

for i in range(3):
    results = env.step(agent='Blue')
    blue_obs = results.observation
    print(blue_obs)

+-----------------+--------------+-------------+----------+-------------+
|      Subnet     |  IP Address  |   Hostname  | Activity | Compromised |
+-----------------+--------------+-------------+----------+-------------+
|  10.0.39.112/28 | 10.0.39.116  |   Defender  |   None   |      No     |
|  10.0.39.112/28 | 10.0.39.125  | Enterprise0 |   None   |      No     |
|  10.0.39.112/28 | 10.0.39.117  | Enterprise1 |   None   |      No     |
|  10.0.39.112/28 | 10.0.39.126  | Enterprise2 |   None   |      No     |
|  10.0.190.64/28 | 10.0.190.76  |   Op_Host0  |   None   |      No     |
|  10.0.190.64/28 | 10.0.190.71  |   Op_Host1  |   None   |      No     |
|  10.0.190.64/28 | 10.0.190.66  |   Op_Host2  |   None   |      No     |
|  10.0.190.64/28 | 10.0.190.73  |  Op_Server0 |   None   |      No     |
| 10.0.174.208/28 | 10.0.174.221 |    User0    |   None   |      No     |
| 10.0.174.208/28 | 10.0.174.222 |    User1    |   None   |      No     |
| 10.0.174.208/28 | 10.0.174.210 |    

The table can also be converted into a vector. This is done by setting the output_mode parameter to 'vector'.

In [7]:
env = BlueTableWrapper(cyborg,output_mode='vector')

env.reset(agent='Blue')
for i in range(3):
    results = env.step(agent='Blue')
    blue_obs = results.observation
    print(blue_obs)
    print(76*'-')

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
----------------------------------------------------------------------------
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
----------------------------------------------------------------------------
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0]
----------------------------------------------------------------------------


The RedTableWrapper is the Red Team version of the BlueTableWrapper.

In [8]:
env = RedTableWrapper(cyborg)

results = env.reset(agent='Red')
print(results.observation)

for i in range(3):
    results = env.step(agent='Red')
    red_obs = results.observation
    print(red_obs)

+---------------+------------+----------+---------+------------+
|     Subnet    | IP Address | Hostname | Scanned |   Access   |
+---------------+------------+----------+---------+------------+
| 10.0.179.0/28 | 10.0.179.1 |  User0   |  False  | Privileged |
+---------------+------------+----------+---------+------------+
+---------------+-------------+-----------------+---------+------------+
|     Subnet    |  IP Address |     Hostname    | Scanned |   Access   |
+---------------+-------------+-----------------+---------+------------+
| 10.0.179.0/28 |  10.0.179.1 |      User0      |  False  | Privileged |
| 10.0.179.0/28 | 10.0.179.11 | UNKNOWN_HOST: 3 |  False  |    None    |
| 10.0.179.0/28 | 10.0.179.14 | UNKNOWN_HOST: 2 |  False  |    None    |
| 10.0.179.0/28 |  10.0.179.2 | UNKNOWN_HOST: 1 |  False  |    None    |
| 10.0.179.0/28 |  10.0.179.5 | UNKNOWN_HOST: 0 |  False  |    None    |
+---------------+-------------+-----------------+---------+------------+
+---------------+-

### PettingZoo Wrapper

Challenge 3 utilises the PettingZoo Wrapper for multiagent reinforcement learning. See Notebook 1. Introduction for how to use it.